# Browser-Use SDK를 사용한 기본 Browser tool 사용법

## 개요

이 튜토리얼에서는 오픈 소스 Browser-Use SDK를 Amazon Bedrock Agentcore Browser tool과 함께 사용하는 방법을 배웁니다. browser tool을 headless 방식으로 사용하고 브라우저를 실시간으로 보는 예제를 제공합니다.


### 튜토리얼 세부 정보


| 정보                | 세부 사항                                                                                   |
|:--------------------|:-------------------------------------------------------------------------------------------        
| 튜토리얼 유형       | 대화형                                                                            |
| Agent 유형          | 단일                                                                                    |
| Agentic Framework   | Browser-Use                                                                               |
| LLM model           | Anthropic Claude 3.7 Sonnet                                                               |
| 튜토리얼 구성 요소  | Browser-Use SDK를 사용하여 Bedrock Agentcore browser tool과 headless 방식으로 상호작용   |
| 튜토리얼 분야       | 범용                                                                            |
| 예제 복잡도         | 쉬움                                                                                      |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK, Browser-Use                                           |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Browser-Use SDK를 Agentcore browser tool과 함께 사용하는 방법을 설명합니다.  

예제에서는 자연어 명령을 Browser-Use agent에 전송하여 Bedrock Agentcore browser에서 headless 방식으로 작업을 수행합니다.


### 튜토리얼 주요 기능

* headless 방식으로 browser tool 사용
* browser tool과 함께 Browser-Use 사용

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

## 작동 방식

browser tool sandbox는 AI agent가 웹 브라우저와 안전하게 상호작용할 수 있도록 하는 보안 실행 환경입니다. 사용자가 요청을 하면 Large Language Model (LLM)이 적절한 tool을 선택하고 명령을 변환합니다. 이러한 명령은 headless 브라우저와 호스팅된 라이브러리 서버(Playwright와 같은 도구 사용)를 포함하는 제어된 sandbox 환경 내에서 실행됩니다. sandbox는 웹 상호작용을 제한된 공간 내에 포함시켜 무단 시스템 액세스를 방지함으로써 격리 및 보안을 제공합니다. agent는 스크린샷을 통해 피드백을 받고 시스템 보안을 유지하면서 자동화된 작업을 수행할 수 있습니다. 

![architecture local](../images/browser-tool.png)

## 1. 환경 설정

#### authorization headers 문제를 해결하고 Amazon Bedrock AgentCore Browser와 호환되도록 하려면 아래 스크립트를 실행하세요

### 1.1 사전 요구 사항 라이브러리 설치 

먼저 browser tool sandbox client에 필요한 라이브러리를 설치합니다. 

In [2]:
!pip install --force-reinstall -U -r requirements.txt --quiet

### 1.2 browser-use를 위한 Patch 스크립트

In [ ]:
%%writefile patch_browser_use.py

#!/usr/bin/env python3
"""Auto-detect and patch browser_use session.py"""

import os
import shutil
import sys
from pathlib import Path

def find_browser_use_path():
    """Automatically find the browser_use installation path"""
    try:
        import browser_use
        # browser_use 패키지의 설치 경로를 찾아서 session.py 파일 위치 반환
        browser_use_path = Path(browser_use.__file__).parent
        session_file = browser_use_path / "browser" / "session.py"
        return str(session_file)
    except ImportError:
        print("❌ browser_use not installed. Install with: pip install browser-use")
        return None

def patch_browser_use():
    # Auto-detect file path
    file_path = find_browser_use_path()
    if not file_path:
        return False
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return False
    
    print(f"📁 Found browser_use at: {file_path}")
    
    # 원본 파일 백업 생성 (한 번만)
    backup_path = file_path + ".backup"
    if not os.path.exists(backup_path):
        shutil.copy2(file_path, backup_path)
        print(f"💾 Created backup: {backup_path}")
    else:
        print(f"📋 Backup already exists: {backup_path}")
    
    # Read file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Bedrock AgentCore와 호환되도록 headers 처리 로직 추가
    old1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True"
    new1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True\n\n\t\tif headers:\n\t\t\tprofile_kwargs['headers'] = headers"
    
    if old1 in content and "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" not in content:
        content = content.replace(old1, new1)
        print("✅ Added headers check")
    elif "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" in content:
        print("✅ Headers check already exists")
    else:
        print("⚠️ Headers check pattern not found")
    
    # CDPClient에 authorization headers 전달하도록 수정
    old2 = "self._cdp_client_root = CDPClient(self.cdp_url)"
    new2 = "self._cdp_client_root = CDPClient(self.cdp_url,  additional_headers=self.browser_profile.headers)"
    
    if old2 in content:
        content = content.replace(old2, new2)
        print("✅ Added headers to CDPClient")
    elif "additional_headers=self.browser_profile.headers" in content:
        print("✅ CDPClient headers already exists")
    else:
        print("⚠️ CDPClient pattern not found")
    
    # Write back
    with open(file_path, 'w') as f:
        f.write(content)
    
    print("🎉 Patching complete!")
    return True

if __name__ == "__main__":
    success = patch_browser_use()
    sys.exit(0 if success else 1)

In [ ]:
# Run the python script to apply the patch for browser-use
!python patch_browser_use.py

In [ ]:
# Restart the Kernal 
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

### 1.3 Import 

 browser tool sandbox client를 초기화하는 데 필요한 라이브러리를 import합니다. 

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use.llm import ChatAnthropicBedrock, ChatAWSBedrock
from browser_use import Agent
from browser_use import Browser, BrowserProfile
from rich.console import Console
from contextlib import suppress
import asyncio

In [2]:
console = Console()

## 2. browser client 설정
browser client를 설정하고 준비될 때까지 기다립니다. 그런 다음 web-socket url과 headers를 생성합니다 
 


In [ ]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

# Bedrock AgentCore browser client 초기화 및 시작
client = BrowserClient(region)
client.start()

# WebSocket URL과 authorization headers 생성
ws_url, headers = client.generate_ws_headers()

## 4. browser 작업을 실행하는 헬퍼 함수
browser-use Agent를 사용하여 브라우저 자동화 작업을 실행합니다 




In [ ]:
async def run_browser_task(browser_session: Browser, bedrock_chat: ChatAnthropicBedrock, task: str) -> None:
    """
    Run a browser automation task using browser_use
    
    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")
        
        # browser_use Agent 생성 (기존 browser session 재사용)
        agent = Agent(
            task=task,
            llm=bedrock_chat,
            browser_session=browser_session
        )
        
        # Run with progress indicator
        with console.status("[bold green]Running browser automation...[/bold green]", spinner="dots"):
            await agent.run()
        
        console.print("[bold green]✅ Task completed successfully![/bold green]")
        
    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback
        if console.is_terminal:
            traceback.print_exc()

## 5. Browser-use profile을 사용하여 browser 작업 호출
CDP WebSocket 연결을 사용하여 지속적인 브라우저 세션을 생성하고 자동화된 웹 작업을 위해 Bedrock Claude model을 초기화합니다. 적절한 정리를 통해 세션 수명 주기를 관리하고 AI 기반 명령을 통해 브라우저 자동화 작업을 실행합니다.

In [ ]:
# Create persistent browser session and model
browser_session = None
bedrock_chat = None

try:
     # authorization headers와 timeout 설정을 포함한 browser profile 생성
    browser_profile = BrowserProfile(
        headers=headers,
        timeout=1500000,  # 150 seconds timeout
    )

    # CDP WebSocket URL로 연결하는 지속적인 browser session 생성
    browser_session = Browser(
        cdp_url=ws_url,
        browser_profile=browser_profile,
        keep_alive=True  # 세션 유지 설정
    )
    
    # Initialize the browser session
    console.print("[cyan]🔄 Initializing browser session...[/cyan]")
    await browser_session.start()
    
    # Bedrock Claude model 초기화 (browser 자동화를 위한 LLM)
    bedrock_chat = ChatAnthropicBedrock(
		model='global.anthropic.claude-haiku-4-5-20251001-v1:0',
		aws_region='us-west-2'
	)
    
    console.print("[green]✅ Browser session initialized and ready for tasks[/green]\n")

    task = "Search for a coffee maker on amazon.com and extract details of the first one" ## Modify the task to run other tasks

    await run_browser_task(browser_session, bedrock_chat, task)

finally:
    # 사용 완료 후 browser session 종료
    if browser_session:
        console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
        with suppress(Exception):
            await browser_session.close()
        console.print("[green]✅ Browser session closed[/green]")


## 6. 정리
브라우저 세션이 아직 중지되지 않았다면 중지합니다

In [ ]:
client.stop()
print("Browser session stopped successfully!")